In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=1

In [2]:
!pip install mat73

DEPRECATION: Loading egg at /Users/maksim/miniconda3/envs/TELF/lib/python3.11/site-packages/rapidfuzz-3.0.0-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
#load data here
import mat73 
import os
X = mat73.loadmat(os.path.join("..", "..", "data", "wtsi.mat"))['X'].astype('float32')
X.shape

(96, 21)

In [4]:
# for testing prunining
import numpy as np
X = np.hstack([X, np.zeros((X.shape[0], 1))])
X = np.vstack([X, np.zeros((1, X.shape[1]))])
X.shape

(97, 22)

In [5]:
from TELF.factorization import NMFk
params = {
    "n_perturbs":36,
    "n_iters":1000,
    "epsilon":0.015,
    "n_jobs":1,
    "init":"nnsvd", 
    "use_gpu":False,
    "save_path":os.path.join("..", "..", "results"),
    "save_output":True,
    "collect_output":True,
    "predict_k":True,
    "verbose":True,
    "transpose":False,
    "sill_thresh":0.9,
    "predict_k_method":"WH_sill",
    "nmf_verbose":False,
    "nmf_method":'nmf_kl_mu',
    'pruned':True,
    "consensus_mat":True,
    "use_consensus_stopping":False,
    "k_search_method":"bst_post",
    "H_sill_thresh":0.1

}
model = NMFk(**params)

Ks = range(1,11,1)

n_perturbs: 36
perturb_type: uniform
n_iters: 1000
epsilon: 0.015
init: nnsvd
save_path: ../../results
save_output: True
use_gpu: False
verbose: True
nmf_verbose: False
perturb_verbose: False
transpose: False
collect_output: True
sill_thresh: 0.9
predict_k: True
predict_k_method: WH_sill
n_jobs: 1
n_nodes: 1
nmf: <function nmf at 0x12a453d80>
nmf_method: nmf_kl_mu
nmf_obj_params: {}
clustering_obj_params: {}
pruned: True
calculate_error: True
consensus_mat: True
use_consensus_stopping: False
mask: None
calculate_pac: False
simple_plot: True
get_plot_data: False
perturb_multiprocessing: False
k_search_method: bst_post
H_sill_thresh: 0.1
factor_thresholding: None
factor_thresholding_H_regression: None
factor_thresholding_obj_params: {}
factor_thresholding_H_regression_obj_params: {}
clustering_method: kmeans
device: [-1]
lock: <unlocked _thread.lock object at 0x3183b39c0>
K_search_settings: {'lock': <unlocked _thread.lock object at 0x30f189040>, 'k_search_method': 'bst_post', 'sill_thres

/Users/maksim/Desktop/Code/telf_internal/TELF/factorization/NMFk.py:856: UserWarning: calculate_error is True! Error calculation can make the runtime longer and take up more memory space!
  warnings.warn(


In [6]:
name = "wtsi"  #name of folder to save resutls
note = "This is an example run of NMFk on wtsi"
results = model.fit(X, Ks, name, note)

/Users/maksim/Desktop/Code/telf_internal/TELF/factorization/NMFk.py:1079: UserWarning: X is data type float64. Whic is not float32. Higher precision will result in significantly longer runtime!
  warnings.warn(


Performing K search with bst_post. Ks=[1, 2, 4, 5, 3, 7, 8, 10, 9, 6]


  0%|          | 0/10 [00:00<?, ?it/s]/Users/maksim/miniconda3/envs/TELF/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/maksim/miniconda3/envs/TELF/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/maksim/miniconda3/envs/TELF/lib/python3.11/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 10/10 [00:05<00:00,  1.77it/s]


In [7]:
print(results.keys())
print('Estimated k=',results['k_predict'])

dict_keys(['time', 'k_predict', 'W', 'H', 'other_results'])
Estimated k= 4


In [8]:
results["W"].shape, results["H"].shape

((97, 4), (4, 22))